In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats/test1.zip
/kaggle/input/dogs-vs-cats/train.zip
/kaggle/input/dogs-vs-cats/sampleSubmission.csv


In [2]:
train_path = "../input/dogs-vs-cats/test1.zip"
test_path = "../input/dogs-vs-cats/train.zip"

files = "/kaggle/working/"

import zipfile

with zipfile.ZipFile(train_path, 'r') as zipp:
    zipp.extractall(files)
    
with zipfile.ZipFile(test_path, 'r') as zipp:
    zipp.extractall(files)

In [3]:
image_dir = "../working/train/"

filenames = os.listdir(image_dir)

labels = [x.split(".")[0] for x in filenames]

data = pd.DataFrame({
     "filename": filenames, 
     "label": labels})
data.head()

image_dir = "../working/test1/"

filenames = os.listdir(image_dir)

labeltest = [x.split(".")[0] for x in filenames]

datatest = pd.DataFrame({
    "id": labeltest,
     "filename": filenames})
datatest.head()

,id,filename
0,10415,10415.jpg
1,4962,4962.jpg
2,1315,1315.jpg
3,11639,11639.jpg
4,12003,12003.jpg


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
plt.figure(figsize=(20,20)) # specifying the overall grid size
plt.subplots_adjust(hspace=0.4)


for i in range(10):
    
    plt.subplot(1,10,i+1)    # the number of images in the grid is 10*10 (100)
    filename = 'train/' + 'cat.' + str(i) + '.jpg'
    image = imread(filename)
    plt.imshow(image)
    plt.title('Cat',fontsize=12)
    plt.axis('off')

plt.show()

In [4]:
from sklearn.model_selection import train_test_split
X=data['filename']
y=data['label']
X_train,X_val=train_test_split(data,random_state=42,stratify=y,test_size=0.2)

In [21]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(rescale=1./255.,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_datagen=ImageDataGenerator(rescale=1./255.)
train_idg = datagen.flow_from_dataframe(X_train, "train/", x_col= "filename", y_col= "label",
                                    batch_size = 64,
                                    target_size=(224,224),class_mode='binary')
val_idg = test_datagen.flow_from_dataframe(X_val,"train/", x_col= "filename", y_col= "label",
                                    batch_size = 64,
                                    target_size=(224,224),class_mode='binary')
test_gen = test_datagen.flow_from_dataframe(datatest,"test1/",x_col= "filename", y_col= None, 
                                          class_mode=None,
                                          batch_size = 64,
                                          target_size = (224,224),
                                          shuffle = False
                                         )
test_gen

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Found 12500 validated image filenames.


In [22]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, InputLayer, Dropout, GlobalAveragePooling2D
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model

In [23]:
from tensorflow.keras.applications.vgg16 import VGG16
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                 input_shape=(224,224,3))
conv_base.trainable=False

In [29]:
x = conv_base.output
x = Flatten()(x)
x=Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x=Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=conv_base.input, outputs=predictions)

In [30]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    metrics=['accuracy'],
    loss='binary_crossentropy'
)
model.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1024)           │    25,691,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,406,849 (154.14 MB)

 Trainable params: 25,692,161 (98.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [31]:
model.fit(train_idg,epochs=10,validation_data=val_idg)

Epoch 1/10


W0000 00:00:1719852460.399068     153 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 720ms/step - accuracy: 0.6209 - loss: 0.7115

W0000 00:00:1719852685.271444     152 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1719852686.593940     153 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


313/313 ━━━━━━━━━━━━━━━━━━━━ 266s 818ms/step - accuracy: 0.6211 - loss: 0.7111 - val_accuracy: 0.8692 - val_loss: 0.3112
Epoch 2/10


W0000 00:00:1719852715.927806     151 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


313/313 ━━━━━━━━━━━━━━━━━━━━ 248s 777ms/step - accuracy: 0.7764 - loss: 0.4605 - val_accuracy: 0.9018 - val_loss: 0.2462
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 248s 776ms/step - accuracy: 0.8078 - loss: 0.4100 - val_accuracy: 0.9052 - val_loss: 0.2289
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 249s 778ms/step - accuracy: 0.8265 - loss: 0.3780 - val_accuracy: 0.9134 - val_loss: 0.2134
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 248s 776ms/step - accuracy: 0.8438 - loss: 0.3502 - val_accuracy: 0.9174 - val_loss: 0.2040
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 248s 777ms/step - accuracy: 0.8503 - loss: 0.3396 - val_accuracy: 0.9204 - val_loss: 0.1979
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 247s 774ms/step - accuracy: 0.8422 - loss: 0.3409 - val_accuracy: 0.9208 - val_loss: 0.1934
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 250s 781ms/step - accuracy: 0.8499 - loss: 0.3314 - val_accuracy: 0.9196 - val_loss: 0.1942
Epoch 9/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 248s 776ms/step - accuracy: 0.8594 - loss: 0.32

In [32]:
test_pred = model.predict(test_gen)
test_pred

196/196 ━━━━━━━━━━━━━━━━━━━━ 69s 352ms/step


array([[0.98990405],
       [0.9999927 ],
       [0.975576  ],
       ...,
       [0.00544524],
       [0.02698863],
       [0.999762  ]], dtype=float32)

In [33]:
y_pred_submit = (test_pred >= 0.5).astype(float)
y_test_submit=y_pred_submit.reshape(1,-1).squeeze()
y_test_submit

array([1., 1., 1., ..., 0., 0., 1.])

In [34]:
result_id=datatest['id']
result = pd.DataFrame({
    'id': result_id,
    'label': y_test_submit
})

In [35]:
result.to_csv("submission.csv", index=False)